# [Day 13: Care Package](https://adventofcode.com/2019/day/13)

## Part 1

As you ponder the solitude of space and the ever-increasing three-hour roundtrip for messages between you and Earth, you notice that the Space Mail Indicator Light is blinking. To help keep you sane, the Elves have sent you a care package.

It's a new game for the ship's arcade cabinet! Unfortunately, the arcade is **all the way** on the other end of the ship. Surely, it won't be hard to build your own - the care package even comes with schematics.

The arcade cabinet runs Intcode software like the game the Elves sent (your puzzle input). It has a primitive screen capable of drawing square **tiles** on a grid. The software draws tiles to the screen with output instructions: every three output instructions specify the `x` position (distance from the left), `y` position (distance from the top), and `tile id`. The `tile id` is interpreted as follows:
- `0` is an **empty** tile. No game object appears in this tile.
- `1` is a **wall** tile. Walls are indestructible barriers.
- `2` is a **block** tile. Blocks can be broken by the ball.
- `3` is a **horizontal paddle** tile. The paddle is indestructible.
- `4` is a **ball** tile. The ball moves diagonally and bounces off objects.

For example, a sequence of output values like `1,2,3,6,5,4` would draw a **horizontal paddle** tile (`1` tile from the left and `2` tiles from the top) and a **ball** tile (`6` tiles from the left and `5` tiles from the top).

Start the game. **How many block tiles are on the screen when the game exits?**

In [9]:
from pathlib import Path

input_path = Path("..") / "inputs" / "day_13.txt"
inputs = input_path.read_text()
inputs = inputs.split(",")
inputs = [int(input_) for input_ in inputs]

In [10]:
def make_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
from collections import Counter

from computer import Computer

In [12]:
%%time
computer = Computer(inputs, [])
computer.run()
results = computer.outputs

chunks = make_chunks(results, 3)
tiles = [chunk[2] for chunk in chunks]
result = Counter(tiles)[2]

CPU times: user 109 ms, sys: 0 ns, total: 109 ms
Wall time: 108 ms


In [13]:
print(f"Answer to Part 1: {result}")

Answer to Part 1: 363


## Part 2

The game didn't run because you didn't put in any quarters. Unfortunately, you did not bring any quarters. Memory address `0` represents the number of quarters that have been inserted; set it to `2` to play for free.

The arcade cabinet has a joystick that can move left and right. The software reads the position of the joystick with input instructions:
- If the joystick is in the **neutral position**, provide `0`.
- If the joystick is **tilted to the left**, provide `-1`.
- If the joystick is **tilted to the right**, provide `1`.

The arcade cabinet also has a segment display capable of showing a single number that represents the player's current score. When three output instructions specify `X=-1, Y=0`, the third output instruction is not a tile; the value instead specifies the new score to show in the segment display. For example, a sequence of output values like `-1,0,12345` would show `12345` as the player's current score.

Beat the game by breaking all the blocks. **What is your score after the last block is broken?**

In [16]:
%%time
inputs[0] = 2
computer = Computer(inputs, [])

results = {}
while True:
    try:
        computer.run()
    except IndexError:
        pass
    chunks = make_chunks(computer.outputs, 3)
    computer.outputs = []
    new_results = {tuple(i[:2]): i[2] for i in chunks}
    results.update(new_results)
    ball_x = [k for k, v in results.items() if v == 4][0][0]
    paddle_x = [k for k, v in results.items() if v == 3][0][0]
    if ball_x > paddle_x:        
        computer.inputs.append(1)
    elif ball_x < paddle_x:
        computer.inputs.append(-1)
    else:
        computer.inputs.append(0)

    if not sum(i == 2 for i in results.values()):
        break
result = results[(-1, 0)]

CPU times: user 4.91 s, sys: 0 ns, total: 4.91 s
Wall time: 4.94 s


In [17]:
print(f"Answer to Part 2: {result} ({result:,})")

Answer to Part 2: 17159 (17,159)
